<a href="https://colab.research.google.com/github/Tamilarasit/Brinjal-Detection/blob/main/Prediction22_06_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csvData=pd.read_csv('/content/drive/MyDrive/My CSV/test.csv')
csvData=csvData.sort_values(['C_Area'],ascending=False)
csvData=csvData.reset_index()
mid=len(csvData)//2
matured_brinjal_Min_val=csvData._get_value(mid,'C_Area')

In [ ]:
dir_path= os.path.join('/content/drive/MyDrive/Sam')
seg_folder=os.path.join('/content/drive/MyDrive/Testing_Segmentation')
identification_folder=os.path.join('/content/drive/MyDrive/Moderately_Matured_Identification')

In [ ]:
for filename in os.listdir(dir_path):
  img=cv2.imread(os.path.join('/content/drive/MyDrive/Testing',filename))
  Ready_to_harvest=img.copy()
  img=cv2.blur(img,(3,3))
  img1=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img2=img1.copy()
  #Preprocessing for Kmeans
  img_RGB=img1
  img_2D_values = img_RGB.reshape((-1,3))
  #Convert to float
  img_val=np.float32(img_2D_values)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.9)
  k=5 # No. of clusters
  retval, labels, centers = cv2.kmeans(img_val, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  centers = np.uint8(centers)
  segmented_data = centers[labels.flatten()]
  segmented_image = segmented_data.reshape((img_RGB.shape))
  centers1=centers.copy()
  max=centers1.argmax(axis=0) #index of the cluster --> Max value represent white area
  x=centers[max[0]]
  centers1[max]=list([0,0,0])
  max_1=centers1.argmax(axis=0)
  max_1
  r=max_1[0]
  centers[r]=x
  for i in range(5):
    if((i!=max[0]) and (i!=max_1[0])):
      centers[i]=list([0,0,0])
  centers = np.uint8(centers)
  segmented_data = centers[labels.flatten()]
  segmented_mask = np.float32(segmented_data.reshape((img_RGB.shape)))
  segmented_mask=cv2.cvtColor(segmented_mask,cv2.COLOR_RGB2GRAY)
  seg_img=segmented_mask.astype(np.uint8)
  kernel = np.ones((3, 3), np.uint8)
  seg_img = cv2.erode(seg_img, kernel, iterations=2)
  ret,binary_img=cv2.threshold(seg_img,100,255,0)
  contrs, hierarchy=cv2.findContours(binary_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
  sum=0
  l=len(contrs)
  for i in range(l):
   #print(cv2.contourArea(contrs[i]))
   sum=sum+cv2.contourArea(contrs[i])
   avg_area=sum/l
   T=avg_area+avg_area//2
   blank_img=np.zeros(seg_img.shape)

   big_index=[]
  for i in range(l):
   if(cv2.contourArea(contrs[i])>T):
      big_index.append(i)
   else:
     cv2.drawContours(blank_img, [contrs[i]], -1, (255,255,255), -1)
  ret,thresh=cv2.threshold(blank_img,100,255,1)
  thresh=thresh.astype(np.uint8)
  brnjal=cv2.bitwise_and(seg_img,seg_img,mask=thresh)
  cont, hierarchy=cv2.findContours(brnjal,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
  '''no_of_contours=len(cont)
  bad_cnt_img=np.zeros(seg_img.shape)
  bad_cnt_index=[]
  for i in range(no_of_contours):
    ellipse = cv2.fitEllipse(cont[i])
    (xc,yc),(d1,d2),angle = ellipse
    if((d1<400) or (d2<400)):
      cv2.drawContours(bad_cnt_img, [cont[i]], -1, (255,255,255), -1)
  ret,bad_cnt_img=cv2.threshold(bad_cnt_img,100,255,1)
  bad_cnt_img=bad_cnt_img.astype(np.uint8)
  brinjals=cv2.bitwise_and(brnjal,brnjal,mask=bad_cnt_img)
  cnt1, hierarchy=cv2.findContours(brinjals,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE
  result=img2.copy()
  i=0
  for c1 in cnt1:
    M=cv2.moments(c1)
    cx=int(M["m10"] / M["m00"])
    cy=int(M["m01"] / M["m00"])
    cv2.circle(result, (int(cx),int(cy)), 20, (255, 0, 0), -1)
    cv2.putText(result, str(i), (int(cx) - 100, int(cy) - 100),cv2.FONT_HERSHEY_SIMPLEX, 5.0, (0, 0, 255), 3)
    i=i+1'''
  perfect_brinjal_img=np.zeros(seg_img.shape)
  for c in cont:
    M0=cv2.moments(c)
    cx0=int(M0["m10"] / M0["m00"])
    cy0=int(M0["m01"] / M0["m00"])
    x_val=c[:,0,0]
    y_val=c[:,0,1]
    a=[]
    b=[]
    for i in range(len(y_val)):
      if (y_val[i]>cy0):
       a.append(x_val[i]/100)
       b.append(y_val[i]/100)
    l=len(a)
    half=l/2
    mid=[]      # Check whether the shape is symmetric or Not
    for i in range(int(half)):
      mid.append((a[i]+a[l-i-1])/2)
    min_val=np.min(mid)
    max_val=np.max(mid)
    dif=np.abs(min_val - max_val)
    if(dif<1):
      cv2.drawContours(perfect_brinjal_img, c, -1, (255,255,255), -1)
  perfect_brinjal_img=perfect_brinjal_img.astype(np.uint8)
  final_brinjals=cv2.bitwise_and(brnjal,brnjal,mask=perfect_brinjal_img)
  cnt2, hierarchy=cv2.findContours(final_brinjals,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
  result1=img2.copy()
  cv2.drawContours(result1, cnt2, -1, (250, 250, 10), 15)
  i=0
  for c1 in cnt2:
    M=cv2.moments(c1)
    cx=int(M["m10"] / M["m00"])
    cy=int(M["m01"] / M["m00"])
    cv2.circle(result1, (int(cx),int(cy)), 20, (255, 0, 0), -1)
    cv2.putText(result1, str(i), (int(cx) - 100, int(cy) - 100),cv2.FONT_HERSHEY_SIMPLEX, 5.0, (0, 0, 255), 3)
    i=i+1
    if (cv2.contourArea(c1)>=matured_brinjal_Min_val):
      cv2.putText(Ready_to_harvest, 'RH', (int(cx) - 100, int(cy) - 100),cv2.FONT_HERSHEY_SIMPLEX, 5.0, (0, 0, 255), 10)
      x,y,w,h=cv2.boundingRect(c1)
      cv2.rectangle(Ready_to_harvest,(x,y),(x+w,y+h),(0,0,255),15)

  cv2.imwrite(seg_folder+'/'+filename,result1)
  cv2.imwrite(identification_folder+'/'+filename,Ready_to_harvest)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')